In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


Model Layers

In [9]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1732,  0.2608,  0.2630, -0.2119,  0.3085,  0.1791,  0.2682, -0.3029,
         -0.5017,  0.0928, -0.2998,  0.5994, -0.3436, -0.1103, -0.1906, -0.4563,
         -0.0514,  0.1149, -0.0822,  0.1180],
        [-0.1984,  0.1038,  0.2098, -0.2114,  0.3294,  0.1016,  0.3956, -0.2513,
         -0.5230,  0.2684, -0.2182,  0.3427, -0.1320, -0.2519,  0.2239, -0.2160,
          0.1633, -0.3365, -0.2125,  0.2422],
        [-0.3038,  0.4958,  0.0742, -0.4463,  0.1475,  0.0720,  0.2241, -0.2220,
         -0.6678,  0.0971, -0.0297,  0.4058, -0.4130,  0.0361, -0.1721, -0.1278,
         -0.2024, -0.1753, -0.0600,  0.1959]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2608, 0.2630, 0.0000, 0.3085, 0.1791, 0.2682, 0.0000, 0.0000,
         0.0928, 0.0000, 0.5994, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1149,
         0.0000, 0.1180],
        [0.0000, 0.1038, 0.2098, 0.0000, 0.3294, 0.1016, 0.3956, 0.0000, 0.0000,
         0.2684, 0.0000, 0.3427, 0.0000, 0.0000, 0.22

In [20]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
#    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0131, -0.0083, -0.0220,  ..., -0.0154, -0.0279, -0.0027],
        [-0.0093, -0.0337, -0.0146,  ..., -0.0158,  0.0025, -0.0171]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0149,  0.0250], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0413,  0.0013,  0.0123,  ..., -0.0349,  0.0298,  0.0129],
        [-0.0364, -0.0333,  0.0041,  ..., -0.0203, -0.0141,  0.0276]],
       device='cuda:0', grad_fn=<Sl

In [21]:
print(f"Model structure: {seq_modules}\n\n")
for name, param in seq_modules.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=20, bias=True)
  (2): Linear(in_features=20, out_features=10, bias=True)
)


Layer: 1.weight | Size: torch.Size([20, 784]) | Values : tensor([[-0.0186,  0.0080, -0.0028,  ..., -0.0015,  0.0193,  0.0147],
        [-0.0028,  0.0255, -0.0332,  ...,  0.0023, -0.0343,  0.0062]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([20]) | Values : tensor([-0.0302,  0.0215], grad_fn=<SliceBackward0>) 

Layer: 2.weight | Size: torch.Size([10, 20]) | Values : tensor([[-0.1191,  0.0496, -0.1117, -0.0336, -0.2097,  0.0006,  0.0924,  0.1212,
         -0.0033, -0.1342,  0.1462, -0.0205,  0.1534, -0.2007,  0.0385,  0.1205,
          0.1974, -0.0928, -0.1968, -0.0181],
        [ 0.2079, -0.2209, -0.0904,  0.1230, -0.1604, -0.1448, -0.1478,  0.2142,
         -0.0675,  0.2043, -0.1386, -0.2074,  0.2186,  0.1218, -0.1195,  0.0854,
          0.1655, -0.1820,  0.1226,  0.2224]],